# geo data

In [1]:
import json

with open('sfpddistricts.geojson.json') as data_file:    
    data = json.load(data_file)

In [2]:
coordinates = []
for index in xrange(10):
    if index == 1: coordinates.append(data['features'][index]['geometry']['coordinates'][0][0])
    else: coordinates.append(data['features'][index]['geometry']['coordinates'][0])

In [3]:
dist_lat = {}
latitudes = []
dist_lon = {}
longitudes = []

for i in range(10):
    longitudes = []
    for j in range(len(coordinates[i])):
        longitudes.append(coordinates[i][j][0])
    dist_lon[i] = longitudes    
    
for i2 in range(10):
    latitudes = []
    for j2 in range(len(coordinates[i2])):
        latitudes.append(coordinates[i2][j2][1])
    dist_lat[i2] = latitudes

In [4]:
def find_max(dic):
    
    max_ini = None
    for key, values in dic.iteritems():
        max_val = max(values)
        
        if max_val > max_ini:
            max_ini = max_val
            
    return max_ini

max_lon = find_max(dist_lon)
max_lat = find_max(dist_lat)

print "max lat: %f" % max_lat
print "max lon: %f" % max_lon

max lat: 37.811575
max lon: -122.356983


In [5]:
def find_min(dic):
    
    min_ini = 0
    for key, values in dic.iteritems():
        min_val = min(values)

        if min_val < min_ini:
            min_ini = min_val
            
    return min_ini

min_lon = find_min(dist_lon)

print "min lon: %f" % min_lon

min lon: -122.514949


## Data to visualize

In [6]:
import pandas as pd

def readfile(filename):
    
    data = pd.read_csv(filename)
    return data

filename = 'SFPD_Incidents_-_from_1_January_2003.csv'
#read file
dataset = readfile(filename)

In [7]:
#extract year from date column in the CSV file and only keep them from 2003 to 2015 (full years in the database)
def filteryears(dataset):
    
    #Create a new column with the extracted years from the date column in the CSV file
    dataset['Year'] = [int(date.split("/")[-1]) for date in dataset['Date']]
    #filter the dataset. Just keep the rows that do not belong to year 2016.
    dataset = dataset[dataset['Year'] != 2016]

    return dataset

db_15 = filteryears(dataset)

In [8]:
#get PROSTITUTION subset

db_PROST = db_15[db_15['Category'] == 'PROSTITUTION']

In [9]:
def geo_data(df):
    
    #get latitudes 
    latitudes = [float(item) for item in df['Y']]
    #get longitudes 
    longitudes = [float(item) for item in df['X']]
    #build X as a list of lists 
    X = [[latitudes[item], longitudes[item]] for item in range(len(latitudes))]
    
    return X

X = geo_data(db_PROST)

In [10]:
districts = list(set(db_PROST['PdDistrict']))
map_dict = {district:index for index, district in enumerate(districts)}

def ylabel(df, map_dict):
    
    y = []    
    for item in df['PdDistrict']:
        y.append(map_dict[item])
        
    return y

Y = ylabel(db_PROST, map_dict)

In [19]:
from sklearn.cross_validation import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [20]:
from sklearn import neighbors

K=3 

def KNN(X,Y,X_test,K):

    #we first create an instance of Neighbours Classifier
    knn=neighbors.KNeighborsClassifier(n_neighbors=K)
    #then we fit the data in order to train the classifier
    knn.fit(X,Y)
    #predict the labels of the coordinates in the grid
    Z = knn.predict(X_test)
    
    return Z

Z = KNN(X_train, Y_train, X_test, K)

In [29]:
import csv

with open('lat_lon.csv', 'w') as fp:
    f = csv.writer(fp, delimiter=',')
    f.writerows(X_test)